In [ ]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")

if not es.indices.exists(index="index-minilm"):
    raise Exception("Index không tồn tại")

mapping_update = {
    "properties": {
        "embedding": {
            "type": "dense_vector",
            "dims": 384,  
            "index": True if es.info()["version"]["number"].startswith("8") else False,
        }
    }
}

es.indices.put_mapping(index="index-minilm", body=mapping_update)

In [2]:
from elasticsearch import Elasticsearch

es = Elasticsearch(["http://localhost:9200"])
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

C:\Users\ABC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from elasticsearch import Elasticsearch

es = Elasticsearch(["http://localhost:9200"])

index_name = "index-minilm"

mapping = {
    "properties": {
        "embedding": {
            "type": "dense_vector",
            "dims": 768,
            "index": True,
            "similarity": "cosine",
            "index_options": {
                "type": "hnsw",
                "m": 16,
                "ef_construction": 100
            }
        }
    }
}

es.indices.put_mapping(index=index_name, body=mapping)

In [ ]:
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

docs = []
scroll = es.search(
    index="index-minilm", 
    scroll="2m", 
    size=1000,
    body={"query": {"match_all": {}}, "_source": ["productname"]},
    request_timeout=60  
)

while len(scroll["hits"]["hits"]) > 0:
    docs.extend(scroll["hits"]["hits"])
    scroll = es.scroll(scroll_id=scroll["_scroll_id"], scroll="2m", request_timeout=60)  

batch_size = 100
for i in tqdm(range(0, len(docs), batch_size)):
    batch = docs[i:i+batch_size]
    
    texts = [doc["_source"]["productname"] for doc in batch]
    embeddings = model.encode(texts, batch_size=batch_size)
    
    bulk_actions = []
    for doc, emb in zip(batch, embeddings):
        bulk_actions.append({
            "update": {
                "_index": "index-minilm",
                "_id": doc["_id"]
            }
        })
        bulk_actions.append({
            "doc": {
                "embedding": emb.tolist()
            }
        })
    
    es.bulk(body=bulk_actions, request_timeout=120)  

In [ ]:
sample = es.search(index="index-minilm", size=1)["hits"]["hits"][0]
print("Document mẫu:", sample["_source"].keys())  

Document mẫu: dict_keys(['productname', 'id', 'embedding'])


In [ ]:
query_text = "cá bống"
query_vector = model.encode(query_text).tolist()

response = es.search(
    index="index-minilm",
    body={
        "knn": {
            "field": "embedding",
            "query_vector": query_vector,
            "k": 10,
            "num_candidates": 100
        },
        "size": 10
    }
)

if response["hits"]["hits"]:
    print(f"Tìm thấy {len(response['hits']['hits'])} kết quả cho '{query_text}': (kNN search)\n")
    for i, hit in enumerate(response["hits"]["hits"], 1):
        print(f"#{i} | Score: {hit['_score']:.4f}")
        print(f"   Tên sản phẩm: {hit['_source'].get('productname', 'N/A')}")
else:
    print("Không tìm thấy kết quả phù hợp")

Tìm thấy 10 kết quả cho 'cá bống': (kNN search)

#1 | Score: 0.9315
   Tên sản phẩm: Vụn cá ngừ Vua Biển túi 1kg (Ngon đỉnh khi làm món cá ngừ chưng mắm tép)
#2 | Score: 0.9195
   Tên sản phẩm: [QUÀ TẶNG] Cá ngừ sốt cay Vissan hộp 170gr
#3 | Score: 0.9148
   Tên sản phẩm: Chà bông cá hồi Chà Bông Việt hũ 40g
#4 | Score: 0.9142
   Tên sản phẩm: Chả Cá Hải Sản Hoa Doanh Gói 60g
#5 | Score: 0.9109
   Tên sản phẩm: Cá Hố (Câu) Đông Lạnh Cà Ná Fish Cắt Khúc khay 300g
#6 | Score: 0.9109
   Tên sản phẩm: [SÁCH] Tư Duy Cá Mập - Suy Nghĩ Cá Vàng
#7 | Score: 0.9079
   Tên sản phẩm: Chả Cá Thát Lát Ốc Vị Sả Ớt Gói 200g
#8 | Score: 0.9055
   Tên sản phẩm: [RẺ] Viên Cá Nhân Trứng Cá Yosa Gói 454g
#9 | Score: 0.9054
   Tên sản phẩm: [MUA 1 TẶNG 1] Cá Ngừ Sốt Cay Vissan hộp 170g
#10 | Score: 0.9051
   Tên sản phẩm: Cá Ngừ Ngâm Dầu Vua Biển hộp 140g


In [7]:
query_text ="cá bống"

response = es.search(
    index="index-minilm",
    body={
        "query": {
            "multi_match": { 
                "query": query_text,
                "fields": ["productname"],  
                "fuzziness": "AUTO" 
            }
        },
        "size": 10  
    }
)
if response["hits"]["hits"]:
    print(f"Tìm thấy {len(response['hits']['hits'])} kết quả cho '{query_text}':(full-text search)\n")
    for i, hit in enumerate(response["hits"]["hits"], 1):
        print(f"#{i} | Score: {hit['_score']:.4f}")
        print(f"   Tên sản phẩm: {hit['_source'].get('productname', 'N/A')}")
else:
    print("Không tìm thấy kết quả phù hợp")

Tìm thấy 10 kết quả cho 'cá bống':(full-text search)

#1 | Score: 9.8842
   Tên sản phẩm: Cá bống mối VAN khay 300gr ( mạ băng 25% giữ độ tươi ngon )
#2 | Score: 9.5597
   Tên sản phẩm: [TẾT] Bánh Orion Con Cá Bống Bang Mochi Socola Đậu Đỏ gói 348g 12P
#3 | Score: 9.5597
   Tên sản phẩm:  Bánh Orion Con Cá Bống Bang Mochi Socola Đậu Đỏ gói 145g (5P*29g)
#4 | Score: 9.5597
   Tên sản phẩm: Bánh con cá Bống Bang Mochi Socola đậu đỏ Orion gói 145gr (5gói*29gr)
#5 | Score: 9.2558
   Tên sản phẩm: [COMBO 3 GÓI] Bánh Orion Con Cá Bống Bang Mochi Socola Đậu Đỏ gói 145g
#6 | Score: 9.2558
   Tên sản phẩm: QT Bánh con cá Bống Bang Mochi Socola đậu đỏ Orion gói 145gr (5gói*29gr)
#7 | Score: 8.8249
   Tên sản phẩm: Cá bống mối XICHMAFOOD khay 500gr +/- 20gr
#8 | Score: 8.7025
   Tên sản phẩm: [ GIẢM 32% COMBO 3 GÓI] Bánh Orion Con Cá Bống Bang Mochi Socola Đậu Đỏ gói 145g
#9 | Score: 8.7025
   Tên sản phẩm: [Siêu Sale] [COMBO 3 GÓI] Bánh Orion Con Cá Bống Bang Mochi Socola Đậu Đỏ gói 145g
#10 | S

In [9]:
def hybrid_search_mini(query_text, index_name="index-minilm", top_k=10):
    try:
        query_embedding = model.encode(query_text, normalize_embeddings=True).tolist()
        

        search_body = {
            "size": top_k,
            "query": {
                "bool": {
                    "should": [
                        {
                            "script_score": {
                                "query": {"match_all": {}},
                                "script": {
                                    "source": "cosineSimilarity(params.query_vector, 'embedding') + 1",
                                    "params": {"query_vector": query_embedding}
                                }
                            }
                        },
                        {
                            "multi_match": {
                                "query": query_text,
                                "boost": 0.06,  
                                "fields": ["productname"],
                                "fuzziness": "AUTO"
                            }
                        }
                    ],
                    "minimum_should_match": 0
                }
            }
        }
        response = es.search(index=index_name, body=search_body)
        
        if response["hits"]["hits"]:
            print(f"Tìm thấy {len(response['hits']['hits'])} kết quả cho '{query_text}':\n")
            for i, hit in enumerate(response["hits"]["hits"], 1):
                print(f"#{i} | Score: {hit['_score']:.4f}")
                print(f"   Tên: {hit['_source'].get('productname', 'N/A')}")
        else:
            print("Không tìm thấy kết quả")

    except Exception as e:
        print(f"Lỗi khi tìm kiếm: {str(e)}")

if __name__ == "__main__":
    hybrid_search_mini("cá bống")

Tìm thấy 10 kết quả cho 'cá bống':

#1 | Score: 2.3180
   Tên: Chà bông cá hồi Chà Bông Việt hũ 40g
#2 | Score: 2.2649
   Tên: Cá bống tẩm ghép oval nướng cán HaiNamFoods khay 70g
#3 | Score: 2.2341
   Tên: Chà bông cá hồi đỏ HaiNamFoods hũ 120g
#4 | Score: 2.2280
   Tên: Cá bống mối VAN khay 300gr ( mạ băng 25% giữ độ tươi ngon )
#5 | Score: 2.1583
   Tên: Cá basa phi lê TPFood khay 500gr ( Bao gồm cả mạ băng )
#6 | Score: 2.1256
   Tên: Cá kho thịt 3 Bông Mai hộp 150gr
#7 | Score: 2.1202
   Tên: [SÁCH] Tư Duy Cá Mập - Suy Nghĩ Cá Vàng
#8 | Score: 2.1098
   Tên: Vụn cá ngừ Vua Biển túi 1kg (Ngon đỉnh khi làm món cá ngừ chưng mắm tép)
#9 | Score: 2.1095
   Tên: [RẺ] Viên Cá Nhân Trứng Cá Yosa Gói 454g
#10 | Score: 2.1051
   Tên: Viên Cá Nhân Trứng Cá Yosa Gói 454g


In [ ]:
def hybrid_search_mini1(query_text, index_name="index-minilm", top_k=10):
    try:
        query_embedding = model.encode(query_text, normalize_embeddings=True).tolist()
        

        search_body = {
            "size": top_k,
            "query": {
                "bool": {
                    "should": [
                        {
                            "script_score": {
                                "query": {"match_all": {}},
                                "script": {
                                    "source": """
                                        if (!doc.containsKey('embedding') || doc['embedding'].size() == 0) return 0;
                                        return cosineSimilarity(params.query_vector, 'embedding') + 1.0
                                    """,
                                    "params": {"query_vector": query_embedding}
                                },
                                "boost": 0.9
                            }
                        },
                        {
                            "match": {
                                "productname": {
                                    "query": query_text,
                                    "fuzziness": "1",
                                    "operator": "OR",
                                    "boost": 0.1
                                }
                            }
                        }
                    ],
                    "minimum_should_match": 1
                }
            },
            # Sửa lỗi rescore query - phiên bản đúng cấu trúc
            "rescore": {
                "window_size": 50,
                "query": {
                    "rescore_query": {
                        "function_score": {
                            "query": {"match_all": {}},
                            "script_score": {
                                "script": {
                                    "source": """
                                        if (!doc.containsKey('embedding') || doc['embedding'].size() == 0) return 0;
                                        return cosineSimilarity(params.query_vector, 'embedding')
                                    """,
                                    "params": {"query_vector": query_embedding}
                                }
                            },
                            "boost_mode": "replace"
                        }
                    },
                    "query_weight": 0.4,
                    "rescore_query_weight": 0.6
                }
            }
        }
        response = es.search(index=index_name, body=search_body)
        
        if response["hits"]["hits"]:
            print(f"Tìm thấy {len(response['hits']['hits'])} kết quả cho '{query_text}':\n")
            for i, hit in enumerate(response["hits"]["hits"], 1):
                print(f"#{i} | Score: {hit['_score']:.4f}")
                print(f"   Tên: {hit['_source'].get('productname', 'N/A')}")
        else:
            print("Không tìm thấy kết quả")

    except Exception as e:
        print(f"Lỗi khi tìm kiếm: {str(e)}")

if __name__ == "__main__":
    hybrid_search_mini1("cá bống")

Tìm thấy 10 kết quả cho 'cá bống':

#1 | Score: 1.4365
   Tên: Cá bống tẩm ghép oval nướng cán HaiNamFoods khay 70g
#2 | Score: 1.4323
   Tên: Chà bông cá hồi Chà Bông Việt hũ 40g
#3 | Score: 1.3642
   Tên: Chà bông cá hồi đỏ HaiNamFoods hũ 120g
#4 | Score: 1.3612
   Tên: Cá basa phi lê TPFood khay 500gr ( Bao gồm cả mạ băng )
#5 | Score: 1.3266
   Tên: Cá Hố (Câu) Đông Lạnh Cà Ná Fish Cắt Khúc khay 300g
#6 | Score: 1.3219
   Tên: Cá bống mối VAN khay 300gr ( mạ băng 25% giữ độ tươi ngon )
#7 | Score: 1.2982
   Tên: Vụn cá ngừ Vua Biển túi 1kg (Ngon đỉnh khi làm món cá ngừ chưng mắm tép)
#8 | Score: 1.2946
   Tên: Cá Bóp Đông lạnh Cà Ná Fish Cắt Khúc khay 500g
#9 | Score: 1.2864
   Tên: [SIÊU SALE] Cá Ngừ Câu Đông Lạnh Cà Ná Fish Nguyên Con 450 - 550g
#10 | Score: 1.2848
   Tên: Cá Ngừ Câu Đông Lạnh Cà Ná Fish Nguyên Con 450 - 550g


In [ ]:
def hybrid_search_mini1_optimized(query_text, index_name="index-minilm", top_k=10):
    try:
        query_embedding = model.encode(query_text, normalize_embeddings=True).tolist()
        
        search_body = {
            "size": top_k,
            "query": {
                "bool": {
                    "should": [
                        {
                            "script_score": {
                                "query": {"exists": {"field": "embedding"}},
                                "script": {
                                    "source": "cosineSimilarity(params.query_vector, 'embedding') + 1.0",
                                    "params": {"query_vector": query_embedding}
                                },
                                "boost": 0.9
                            }
                        },
                        {
                            "multi_match": {
                                "query": query_text,
                                "fields": ["productname"],
                                "fuzziness": "1",
                                "operator": "or",
                                "boost": 0.1
                            }
                        }
                    ],
                    "minimum_should_match": 1
                }
            },
           
            "rescore": {
                "window_size": top_k * 2,
                "query": {
                    "rescore_query": {
                        "script_score": {
                            "query": {"exists": {"field": "embedding"}},
                            "script": {
                                "source": "cosineSimilarity(params.query_vector, 'embedding')",
                                "params": {"query_vector": query_embedding}
                            }
                        }
                    },
                    "query_weight": 0.4,
                    "rescore_query_weight": 0.6
                }
            }
        }
        
        response = es.search(
            index=index_name, 
            body=search_body,
            request_timeout=30  
        )
        
        if response["hits"]["hits"]:
            print(f"Tìm thấy {len(response['hits']['hits'])} kết quả cho '{query_text}':\n")
            for i, hit in enumerate(response["hits"]["hits"], 1):
                print(f"#{i} | Score: {hit['_score']:.4f}")
                print(f"   Tên: {hit['_source'].get('productname', 'N/A')}")
        else:
            print("Không tìm thấy kết quả")

    except Exception as e:
        print(f"Lỗi khi tìm kiếm: {str(e)}")

if __name__ == "__main__":
    hybrid_search_mini1_optimized("cá bống")

Tìm thấy 10 kết quả cho 'cá bống':

#1 | Score: 1.4365
   Tên: Cá bống tẩm ghép oval nướng cán HaiNamFoods khay 70g
#2 | Score: 1.4323
   Tên: Chà bông cá hồi Chà Bông Việt hũ 40g
#3 | Score: 1.3642
   Tên: Chà bông cá hồi đỏ HaiNamFoods hũ 120g
#4 | Score: 1.3612
   Tên: Cá basa phi lê TPFood khay 500gr ( Bao gồm cả mạ băng )
#5 | Score: 1.3266
   Tên: Cá Hố (Câu) Đông Lạnh Cà Ná Fish Cắt Khúc khay 300g
#6 | Score: 1.3219
   Tên: Cá bống mối VAN khay 300gr ( mạ băng 25% giữ độ tươi ngon )
#7 | Score: 1.2946
   Tên: Cá Bóp Đông lạnh Cà Ná Fish Cắt Khúc khay 500g
#8 | Score: 1.2864
   Tên: [SIÊU SALE] Cá Ngừ Câu Đông Lạnh Cà Ná Fish Nguyên Con 450 - 550g
#9 | Score: 1.2848
   Tên: Cá Ngừ Câu Đông Lạnh Cà Ná Fish Nguyên Con 450 - 550g
#10 | Score: 1.2601
   Tên: Cá kho thịt 3 Bông Mai hộp 150gr


C:\Users\ABC\AppData\Local\Temp\ipykernel_16356\103243297.py:56: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  response = es.search(
